Import library

In [2]:
from lxml import etree as ET
import pandas as pd
from configparser import ConfigParser
import psycopg2
from config import config

In [2]:
ELEMENTS = ['article', 'inproceedings', 'proceedings', 'book', 'incollection', 'phdthesis', 'mastersthesis', 'www']

FEATURES = ['author', 'title', 'school', 'year', 'pages', 'ee', 'publisher', 'booktitle', 'address', 'journal', 'volume', 
            'number', 'month', 'url', 'cdrom', 'editor', 'cite', 'note', 'crossref', 'isbn', 'series', 'chapter', 'publnr']

Handle coincided features

In [3]:
def handle_coincided(tag, text, value):
    tag_cp = tag.copy()
    text_cp = text.copy()
    count = tag_cp.count(value)
    index = tag_cp.index(value)

    list_connect = []
    for i in range(index, index + count):
        list_connect.append(text_cp[i])
    
    list_connect = ', '.join(list_connect)
    for i in range(count - 1):
        tag_cp.remove(value)
    
    for i in range(index, index + count):
        text_cp.remove(text[i])
    
    text_cp.insert(index, list_connect)
    return tag_cp, text_cp

Read file XML 

In [4]:
def getFeature(value, num_features):
    tree = ET.iterparse('./dblp.xml', dtd_validation=True, load_dtd=True)
    tags = []
    texts = []
    count = 0
    df = pd.DataFrame(columns = FEATURES)

    for _, element in tree:
        if(element.tag in FEATURES):
            tags.append(element.tag)
            texts.append(element.text)
        elif(element.tag in ELEMENTS):
            if(element.tag == value):
                try:
                    tags.append('ELM')
                    texts.append(element.tag)

                    unique = list(set(tags))
                    for i in unique:
                        if(tags.count(i) > 1):
                            tags, texts = handle_coincided(tags, texts, i)

                    arr = []
                    for i in range(len(FEATURES)):
                        if(FEATURES[i] in tags):
                            index = tags.index(FEATURES[i])
                            arr.append(texts[index])
                        else:
                            arr.append(None)
                    keys = FEATURES
                    items = arr
                    data = dict(zip(keys, items))
                    df = df.append(data, ignore_index=True)
                    count = count + 1

                except:
                    pass
                
                if(count % 1000 == 0):
                    print(count)

                if(count == num_features): 
                    break

            tags = []
            texts = []
        
        element.clear()
        while element.getprevious() is not None:
            del element.getparent()[0]  
    return df

In [26]:
df_final = getFeature('mastersthesis', 10000)

In [27]:
df_final

,author,title,school,year,pages,ee,publisher,booktitle,address,journal,...,url,cdrom,editor,cite,note,crossref,isbn,series,chapter,publnr
0,Tatu Ylönen,Shadow Paging Is Feasible.,"Helsinki University of Technology, Department ...",1994,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Peter Van Roy,A Prolog Compiler for the PLM.,University of California at Berkeley,1984,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Salim Perchy,Multimedia Interaction with NTCC. (Interacción...,"Pontifical Xavierian University, Bogotá, Colombia",2013,None,https://tel.archives-ouvertes.fr/tel-01257184,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Christian Schulte 0001,Entwurf und Implementierung eines übersetzende...,"Universität Karlsruhe, Institut für Logik, Kom...",1991,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Satyan R. Coorg,Partitioning non-strict languages for multi-th...,"Massachusetts Institute of Technology, Cambrid...",1994,None,http://hdl.handle.net/1721.1/32607,None,None,None,None,...,None,None,None,None,ndltd.org (oai:dspace.mit.edu:1721.1/32607),None,None,None,None,None
5,Rodrigo Pastl Pontes,Contribuições do model checking e da metodolog...,"Instituto Tecnológico de Aeronáutica, São José...",2011,None,http://www.bd.bibl.ita.br/tde_busca/arquivo.ph...,None,None,None,None,...,None,None,None,None,ndltd.org (oai:agregador.ibict.br.BDTD_ITA:oai...,None,None,None,None,None
6,Salma Hosni Emam Mohamed Elmalaki,Context-Aware Runtime Engine For Android Opera...,"University of California, Los Angeles, USA",2014,None,"http://www.escholarship.org/uc/item/2f74g5mw, ...",None,None,None,None,...,None,None,None,None,base-search.net (ftcdlib:qt2f74g5mw),None,None,None,None,None
7,Vanessa C. Klaas,Who's Who in the World Wide Web: Approaches to...,"Diplomarbeit, LMU München, Informatik",2007,None,http://www.pms.ifi.lmu.de/publikationen/diplom...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Kurt P. Brown,PRPL: A Database Workload Specification Langua...,University of Wisconsin-Madison,1992,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Rita Ley,Der Einfluss kleiner naturnaher Retentionsmaßn...,"Diplomarbeit, Universität Trier, FB VI, Physis...",2006,None,http://dblp.uni-trier.de/papers/DiplomarbeitRi...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Save data to file csv

In [28]:
df_final.to_csv('mastersthesis_data.csv', index = False)

Save file csv to PosgreSQL 

Connect to PostgreSQL database server and save CSV to server

In [5]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        conn = psycopg2.connect(database="DBLP",
                                user='postgres', password='duyduyduc18102001', 
                                host='localhost', port='5432'
        )
        conn.autocommit = True  

        # create a cursor
        cur = conn.cursor()
        
	    # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

        sql = '''CREATE TABLE BOOK(
                author   varchar(10000),
                title   varchar(10000),
                school   varchar(10000),
                year   varchar(10000),
                pages   varchar(10000),
                ee   varchar(10000),
                publisher   varchar(10000),
                booktitle   varchar(10000),
                address   varchar(10000),
                journal   varchar(10000),
                volume   varchar(10000),
                number   varchar(10000),
                month   varchar(10000),
                url   varchar(10000),
                cdrom   varchar(10000),
                editor   varchar(10000),
                cite   varchar(10000),
                note   varchar(10000),
                crossref   varchar(10000),
                isbn   varchar(10000),
                series   varchar(10000),
                chapter   varchar(10000),
                publnr   varchar(10000));'''

        cur.execute(sql)

        sql2 = '''COPY book(
                author,
                title,
                school,
                year,
                pages,
                ee,
                publisher,
                booktitle,
                address,
                journal,
                volume,
                number,
                month,
                url,
                cdrom,
                editor,
                cite,
                note,
                crossref,
                isbn,
                series,
                chapter,
                publnr)
                FROM 'D:\\Hoc_ky_6\Tinh_Toan_Da_Phuong_Tien\\dblp\\book_data.csv'
                DELIMITER ','
                CSV HEADER;'''

        cur.execute(sql2)
        
	    # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.
